In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import os
import glob
import shutil



path="example-p0.30000000000000004;a2.8e-07;l1.8000000000000002e-07;s0.out"

def loadFile(name):
    data=pd.read_csv(name,sep="\t")
    data["H"]=np.round(data["B_extx (T)"]*10000/(np.sqrt(2)/2))
    data["m_full"]=(data["m_fullx (A/m)"]+data["m_fully (A/m)"])/np.sqrt(2)
    data["m"]=data["m_full"]/np.max(np.abs(data["m_full"]))

    try: 
        data["hIsland_my"]=data["m_full.region1y (A/m)"]/np.max(np.sqrt(data["m_full.region1y (A/m)"]**2+data["m_full.region1x (A/m)"]**2))
    except Exception:
        pass
    

    data=data.drop(columns=["# t (s)",\
    "mz ()","mx ()", "my ()",\
    "B_extx (T)", "B_exty (T)","B_extz (T)",\
    "m_fullz (A/m)","m_fullx (A/m)","m_fully (A/m)",\
    "m_full.region1x (A/m)","m_full.region1y (A/m)","m_full.region1z (A/m)",\
    "m_full.region2x (A/m)","m_full.region2y (A/m)","m_full.region2z (A/m)"])

    try:
        data.attrs["Hc"]=np.average(np.abs(data[np.sign(data["m"]).diff() != 0]["H"].iloc[1:]))
        data.attrs["Br"]=np.average(np.abs(data[np.sign(data["H"]).diff() != 0]["m"].iloc[1:]))
    except Exception:
        pass


    return data

imagePaths=sorted(glob.glob(path+"/*.png"))[1:]
images=[]
data=loadFile(path+"/table.txt")

for imPath in imagePaths:
    im = plt.imread(imPath)
    images.append(im)

#Load "images" and "data"


In [2]:
assert len(images)==len(data)



#Make temporary directory
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'Temp Files')
if os.path.exists(final_directory):
   shutil.rmtree(final_directory)
os.makedirs(final_directory)


In [3]:



for i in range(len(images)):

   #if i%200!=0: continue
   print(i)

   fig,(imAx,pltAx)=plt.subplots(1,2)
   fig.set_size_inches(13, 7)
   fig.set_dpi(200)

   thisRow=data.loc[i,:]

   imAx.imshow(images[i],interpolation='nearest', aspect='auto')
   imAx.invert_yaxis()
   imAx.set_xlabel("x (nm)",fontsize=20)
   imAx.set_ylabel("y (nm)",fontsize=20)

   arrowScale=thisRow.H/max(data.H)

   imAx.arrow(0.5,0.5,0.3*arrowScale,0.3*arrowScale,width=0.1,head_length=0.1,lw=0,color="white",head_width=0.15,alpha=0.5,transform=imAx.transAxes)
   imAx.text(0.5,0.5,f"{round(thisRow.H)}Oe",horizontalalignment='center',
        verticalalignment='center',rotation=45,fontsize=20,fontweight=2,transform=imAx.transAxes)


   pltAx.plot(data["H"],data["m"],lw=3,color="k")
   pltAx.plot(-data["H"],-data["m"],lw=3,color="k")

   
   pltAx.scatter(thisRow["H"],thisRow["m"],s=200,zorder=10,color="red")
   pltAx.set_xlabel("H (Oe)", fontsize=20)
   pltAx.set_ylabel("$M$/$M_s$",fontsize=20)

   plt.suptitle("$p=0.8$\n$a=280nm$\n$\ell=180nm$",fontsize=20)

   pltAx.tick_params(axis='x', labelsize=20)
   pltAx.tick_params(axis='y', labelsize=20)
   imAx.tick_params(axis='x', labelsize=20)
   imAx.tick_params(axis='y', labelsize=20)
   
   plt.tight_layout()
   plt.savefig(final_directory+f"/{i}.png")
   plt.close()



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
os.system(f"ffmpeg -framerate 30  -i '{final_directory}/%d.png' \
  -c:v libx264 -pix_fmt yuv420p out.mp4 -y")

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

0

: 